In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [2]:
def network_setting(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg ==0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg ==0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component)<10:
            for node in component:
                _nodes_removed+=1
                network.remove_node(node)
    for u, v, k, data in tqdm(G.edges(data=True, keys=True),position=0):
        if 'maxspeed' in data.keys():
            speed_type = type(data['maxspeed'])
            if (speed_type==str):
                if len(data['maxspeed'].split(','))==2:
                    data['maxspeed']=float(data['maxspeed'].split(',')[0])                  
                elif data['maxspeed']=='signals':
                    data['maxspeed']=35.0 # drive speed setting as 35 miles
                else:
                    data['maxspeed']=float(data['maxspeed'].split()[0])
            else:
                data['maxspeed']=float(data['maxspeed'][0].split()[0])
        else:
            data['maxspeed']= 35.0 #miles
        data['maxspeed_meters'] = data['maxspeed']*26.8223 # convert mile to meter
        data['time'] = float(data['length'])/ data['maxspeed_meters']
    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed/float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))
    return(network)

def hospital_setting(hospitals, G):
    hospitals['nearest_osm']=None
    for i in tqdm(hospitals.index, desc="Find the nearest osm from hospitals", position=0):
        hospitals['nearest_osm'][i] = ox.get_nearest_node(G, [hospitals['Y'][i], hospitals['X'][i]], method='euclidean') # find the nearest node from hospital location
    print ('hospital setting is done')
    return(hospitals)

In [3]:
%%time
G = ox.load_graphml('Illinois_Network.graphml', folder="Data", node_type=str)
G = network_setting (G)
hospitals = gpd.read_file('./Data/HospitalData/{}_Hospital_Info.shp'.format("Illinois"))
hospitals = hospital_setting(hospitals, G)
distances=[10,20,30]

100%|███████████████████████████████████████████████| 1588320/1588320 [00:04<00:00, 391918.52it/s]


Removed 713 nodes (0.0012%) from the OSMNX network
Number of nodes: 593314


Find the nearest osm from hospitals:   0%|                                | 0/183 [00:00<?, ?it/s]

Number of edges: 1588320


Find the nearest osm from hospitals: 100%|██████████████████████| 183/183 [07:26<00:00,  2.44s/it]

hospital setting is done
Wall time: 11min 37s


In [4]:
def dijkstra_cca(G, nearest_osm, distance, distance_unit = "time"):
    road_network = G.subgraph(nx.single_source_dijkstra_path_length(G, nearest_osm, distance, distance_unit))  
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

In [5]:
def ego_cca(G, nearest_osm, distance, distance_unit = "time"):
    road_network = nx.ego_graph(G, nearest_osm, distance, distance=distance_unit) 
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

In [6]:
%%time
ego_cca(G, hospitals['nearest_osm'][0], distances[2])

Wall time: 4.67 s


,geometry
0,"POLYGON ((-90.08384 38.60189, -90.08789 38.601..."


In [7]:
%%time 
dijkstra_cca(G, hospitals['nearest_osm'][0], distances[2])

Wall time: 880 ms


,geometry
0,"POLYGON ((-90.08384 38.60189, -90.08789 38.601..."


In [8]:
%%time
ego = []
for i in range(10):
    ego.append(ego_cca(G, hospitals['nearest_osm'][i], distances[2]))

Wall time: 28.2 s


In [9]:
%%time
dij = []
for i in range(10):
    dij.append( dijkstra_cca(G, hospitals['nearest_osm'][i], distances[2]))

Wall time: 11 s


In [10]:
for i in range(10):
    print(ego[i].geometry == dij[i].geometry)

0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
